In [29]:

!rm -rf /root/nltk_data
!pip install --upgrade --force-reinstall nltk --quiet
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
print("✅ All NLTK resources downloaded")



✅ All NLTK resources downloaded


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [31]:
w = pattern.lower().split()



In [34]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [36]:
import nltk
nltk.download('wordnet', download_dir='/root/nltk_data')
nltk.download('omw-1.4', download_dir='/root/nltk_data')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [37]:
import os
nltk.data.path.append('/root/nltk_data')


In [47]:
import json
import pickle

words = []
classes = []
documents = []
ignore_words = ['?', '!']
with open("/content/intents.json") as file:
    intents = json.load(file)
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = pattern.lower().split()
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = sorted(set(w for w in words if w not in ignore_words))
classes = sorted(set(classes))
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

print("✅ Preprocessing complete")
print(f"Documents: {len(documents)}")
print(f"Classes: {classes}")
print(f"Words: {words[:10]} ... total: {len(words)}")


✅ Preprocessing complete
Documents: 64
Classes: ['cancel_order', 'change_password', 'contact_support', 'discounts', 'goodbye', 'greeting', 'location', 'make_payment', 'open_hours', 'order_status', 'product_info', 'refund_status', 'return_policy', 'technical_support', 'thanks']
Words: ['a', 'about', 'accept', 'address?', 'agent', 'any', 'app', 'appreciate', 'are', 'available'] ... total: 115


In [48]:
import numpy as np
import random
import pickle
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = [w.lower() for w in doc[0]]
    for w in words:
        bag.append(1 if w in pattern_words else 0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print("✅ Training data ready")
print(f"train_x shape: {train_x.shape}")
print(f"train_y shape: {train_y.shape}")


✅ Training data ready
train_x shape: (64, 115)
train_y shape: (64, 15)


In [49]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5")

print("✅ Model trained and saved")


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0561 - loss: 2.7185   
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1120 - loss: 2.7033 
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1965 - loss: 2.6464 
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2270 - loss: 2.6407     
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1808 - loss: 2.5882 
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1641 - loss: 2.5806     
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1683 - loss: 2.5171     
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3547 - loss: 2.4051 
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3483 - loss: 2.3302 
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3446 - loss: 2.3526 
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4840 - loss: 2.0522 
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/ste

✅ Model trained and saved


In [50]:
import json
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.load(open('intents.json'))
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))


In [51]:
def bag_of_words(sentence, words):
    sentence_words = sentence.lower().split()
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [
        {"intent": classes[i], "probability": str(prob)}
        for i, prob in enumerate(res)
        if prob > ERROR_THRESHOLD
    ]
    results.sort(key=lambda x: float(x["probability"]), reverse=True)
    return results

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    for intent in intents_json['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    return "Sorry, I didn't understand that."

def chatbot_response(msg):
    ints = predict_class(msg)
    if ints:
        return get_response(ints, intents)
    else:
        return "Sorry, I didn't understand that."


In [52]:
print("🤖 Chatbot is ready! Type 'quit' to stop.")

while True:
    msg = input("You: ")
    if msg.lower() in ['quit', 'exit']:
        print("Bot: Goodbye!")
        break
    print("Bot:", chatbot_response(msg))


🤖 Chatbot is ready! Type 'quit' to stop.
You: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Bot: Hi there! What can I help you with?
You: exit
Bot: Goodbye!
